<a href="https://colab.research.google.com/github/philnumpy/PRML-PROJECT/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [3]:
data=pd.read_csv('spotify_millsongdata.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [6]:
data=data.drop_duplicates()

In [8]:
data.size

230600

In [9]:
data.shape

(57650, 4)

In [10]:
for col in data.columns:
  data.dropna(inplace=True)

In [25]:
df=data.sample(n=5000, random_state=42)

In [27]:
print(df)

             artist                             song  \
56679  Wishbone Ash                   Right Or Wrong   
224       Aerosmith        This Little Light Of Mine   
32457  Fall Out Boy                     Dance, Dance   
9167   Janis Joplin                       Easy Rider   
13270   Moody Blues                        Peak Hour   
...             ...                              ...   
15274       Outkast                        Prototype   
821    Arlo Guthrie           Ridin' Down The Canyon   
23364  Alice Cooper  Can't Sleep, Clowns Will Eat Me   
48860        Poison           Emperors's New Clothes   
13695  Neil Diamond                            Crazy   

                                                    link  \
56679       /w/wishbone+ash/right+or+wrong_20147150.html   
224    /a/aerosmith/this+little+light+of+mine_2064448...   
32457          /f/fall+out+boy/dance+dance_10113666.html   
9167            /j/janis+joplin/easy+rider_10147381.html   
13270             /m/moody+

In [13]:
song_names=df['song']

In [14]:
pip install spotipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 5.4 MB/s eta 0:00:00


In [28]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

# Set up API authentication
client_id = "bc9b2f9012004d50819cc568182c9c34"
client_secret = "b3886413986340d1b7a8885c1ed32cd7"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Function to get track features with retry logic
def get_audio_features(track_name, artist_name):
    query = f"track:{track_name} artist:{artist_name}"

    try:
        results = sp.search(q=query, type="track", limit=1)

        if results["tracks"]["items"]:
            track_id = results["tracks"]["items"][0]["id"]  # Get the track ID

            # Handle rate limits
            while True:
                try:
                    features = sp.audio_features([track_id])[0]  # Fetch audio features
                    return features
                except spotipy.exceptions.SpotifyException as e:
                    if e.http_status == 403:  # Rate limit error
                        retry_after = int(e.headers.get("Retry-After", 5))  # Get wait time
                        print(f"Rate limited. Waiting {retry_after} seconds...")
                        time.sleep(retry_after)
                    else:
                        print(f"Error fetching features for track ID {track_id}: {e}")
                        return None
        else:
            return None  # No match found

    except Exception as e:
        print(f"Unexpected error for {track_name} - {artist_name}: {e}")
        return None



# Save progress every 500 songs
batch_size = 500
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    batch["audio_features"] = batch.apply(lambda row: get_audio_features(row["song"], row["artist"]), axis=1)

    # Drop rows where 'audio_features' is None
    batch = batch.dropna(subset=['audio_features'])
    audio_features_df = batch["audio_features"].apply(pd.Series)
    batch = pd.concat([batch, audio_features_df], axis=1).drop(columns=["audio_features"])

    # Save to CSV (append mode)
    batch.to_csv("merged_spotify_data.csv", mode="a", header=(i == 0), index=False)
    print(f"Saved batch {i+1} to {i+batch_size}")

    time.sleep(2)  # Wait 2 seconds between batches

print("Dataset updated with Spotify audio features!")


ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=03nBg3SzfdFzcIr6XikvCs with Params: {} returned 403 due to None


Rate limited. Waiting 5 seconds...


KeyboardInterrupt: 